In [2]:
import os
import io
import gradio as gr
from IPython.display import Markdown, display, update_display
import subprocess
import sys

In [5]:
model="lm_studio/medgemma-4b-it"

In [4]:
from openai import OpenAI

llm = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio"
)
llm

In [6]:
system_message = (
    "You are an assitant that reimplements Python code in high performance C++ for M1 Mac."
    "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments."
    "The C++ response needs to produce an identical output in the fastest possible time."
)
system_message

'You are an assitant that reimplements Python code in high performance C++ for M1 Mac.Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments.The C++ response needs to produce an identical output in the fastest possible time.'

In [7]:
def user_prompt_for(python):
    user_prompt = (
        "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time."
        "Respond only with C++ code; do not explain your work other than a few comments."
        "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
        f"{python}"
    )
    return user_prompt


In [8]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [9]:
def write_output(cpp):
    code = cpp.replace("```cpp", "").replace("```", "")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [10]:
def optimize_code(python):
    stream = llm.chat.completions.create(
        model=model,
        messages=messages_for(python),
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end="", flush=True)
    write_output(reply)

In [11]:
python_code = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""
python_code

'\nimport time\n\ndef calculate(iterations, param1, param2):\n    result = 1.0\n    for i in range(1, iterations+1):\n        j = i * param1 - param2\n        result -= (1/j)\n        j = i * param1 + param2\n        result += (1/j)\n    return result\n\nstart_time = time.time()\nresult = calculate(100_000_000, 4, 1) * 4\nend_time = time.time()\n\nprint(f"Result: {result:.12f}")\nprint(f"Execution Time: {(end_time - start_time):.6f} seconds")\n'

In [12]:
exec(python_code)

Result: 3.141592658589
Execution Time: 9.553332 seconds


In [13]:
optimize_code(python_code)

```cpp
#include <iostream>
#include <iomanip>
#include <cmath>

double calculate(long long iterations, double param1, double param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= 1.0 / j;
        j = i * param1 + param2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    long long iterations = 100000000LL;
    double param1 = 4.0;
    double param2 = 1.0;
    double result = calculate(iterations, param1, param2) * 4.0;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << (time_point_cast<chrono::seconds>(std::chrono::high_resolution_clock::now()).time_since_epoch().count() - time_point_cast<chrono::seconds>(std::chrono::high_resolution_clock::now()).time_since_epoch().count()) / 1000000.0 << " seconds" << std::endl;

    return 0;
}
```

In [14]:
def stream_code(python):
    stream = llm.chat.completions.create(
        model=model,
        messages=messages_for(python),
        stream=True
    )
    reply = ""

    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace("```cpp\n", "").replace("```", "")

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code", lines=10, value=python_code)
        cpp = gr.Textbox(label="C++ Code: ", lines=10)
    with gr.Row():
        # model = gr.Dropdown()
        convert = gr.Button("Convert Code")
    
    convert.click(optimize_code, inputs=[python], outputs=[cpp])
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


```cpp
#include <iostream>
#include <iomanip>
#include <cmath>

double calculate(long long iterations, double param1, double param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= 1.0 / j;
        j = i * param1 + param2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    long long iterations = 100000000LL;
    double param1 = 4.0;
    double param2 = 1.0;
    double result = calculate(iterations, param1, param2) * 4.0;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << (time_point_cast<chrono::seconds>(std::chrono::high_resolution_clock::now() - time_point_cast<chrono::seconds>(start_time))).count() / 1000.0 << " seconds" << std::endl;

    return 0;
}
``````cpp
#include <iostream>
#include <iomanip>
#include <cmath>

double calculate(long long iterations, double param1, double param2